In [10]:
import layers

In [11]:
import numpy as np
import torch
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F

from layers import GraphConvolution, MLP


device = f"cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
large_datasets = [
    "deezer-europe",
    "yelp-chi",
    "Penn94",
    "arxiv-year",
    "pokec",
    "snap-patents",
    "genius",
    "twitch-gamer",
    "wiki",
]


class GCN(nn.Module):
    def __init__(
        self,
        nfeat,
        nhid,
        nclass,
        nlayers,
        nnodes,
        dropout,
        model_type,
        structure_info,
        variant=False,
        init_layers_X=1,
    ):
        super(GCN, self).__init__()
        if model_type == "acmgcnpp":
            self.mlpX = MLP(nfeat, nhid, nhid, num_layers=init_layers_X, dropout=0)
        self.gcns, self.mlps = nn.ModuleList(), nn.ModuleList()
        self.model_type, self.structure_info, self.nlayers, self.nnodes = (
            model_type,
            structure_info,
            nlayers,
            nnodes,
        )

        if (
            self.model_type == "acmgcn"
            or self.model_type == "acmgcnp"
            or self.model_type == "acmgcnpp"
        ):
            self.gcns.append(
                GraphConvolution(
                    nfeat,
                    nhid,
                    nnodes,
                    model_type=model_type,
                    variant=variant,
                    structure_info=structure_info,
                )
            )
            self.gcns.append(
                GraphConvolution(
                    1 * nhid,
                    nclass,
                    nnodes,
                    model_type=model_type,
                    output_layer=1,
                    variant=variant,
                    structure_info=structure_info,
                )
            )
        elif self.model_type == "acmsgc":
            self.gcns.append(GraphConvolution(nfeat, nclass, nnodes, model_type=model_type))
        elif self.model_type == "acmsnowball":
            for k in range(nlayers):
                self.gcns.append(
                    GraphConvolution(
                        k * nhid + nfeat, nhid, nnodes, model_type=model_type, variant=variant
                    )
                )
            self.gcns.append(
                GraphConvolution(
                    nlayers * nhid + nfeat,
                    nclass,
                    nnodes,
                    model_type=model_type,
                    variant=variant,
                )
            )
        self.dropout = dropout
        self.fea_param, self.xX_param = Parameter(
            torch.FloatTensor(1, 1).to(device)
        ), Parameter(torch.FloatTensor(1, 1).to(device))

        self.reset_parameters()

    def reset_parameters(self):
        if self.model_type == "acmgcnpp":
            self.mlpX.reset_parameters()
        else:
            pass

    def forward(self, x, adj_low, adj_high, adj_low_unnormalized):

        if (
            self.model_type == "acmgcn"
            or self.model_type == "acmsgc"
            or self.model_type == "acmsnowball"
            or self.model_type == "acmgcnp"
            or self.model_type == "acmgcnpp"
        ):

            x = F.dropout(x, self.dropout, training=self.training)
            if self.model_type == "acmgcnpp":
                xX = F.dropout(
                    F.relu(self.mlpX(x, input_tensor=True)),
                    self.dropout,
                    training=self.training,
                )
        if self.model_type == "acmsnowball":
            list_output_blocks = []
            for layer, layer_num in zip(self.gcns, np.arange(self.nlayers)):
                if layer_num == 0:
                    list_output_blocks.append(
                        F.dropout(
                            F.relu(layer(x, adj_low, adj_high)),
                            self.dropout,
                            training=self.training,
                        )
                    )
                else:
                    list_output_blocks.append(
                        F.dropout(
                            F.relu(
                                layer(
                                    torch.cat([x] + list_output_blocks[0:layer_num], 1),
                                    adj_low,
                                    adj_high,
                                )
                            ),
                            self.dropout,
                            training=self.training,
                        )
                    )
            return self.gcns[-1](
                torch.cat([x] + list_output_blocks, 1), adj_low, adj_high
            )

        fea1 = self.gcns[0](x, adj_low, adj_high, adj_low_unnormalized)

        if (
            self.model_type == "acmgcn"
            or self.model_type == "acmgcnp"
            or self.model_type == "acmgcnpp"
        ):

            fea1 = F.dropout((F.relu(fea1)), self.dropout, training=self.training)

            if self.model_type == "acmgcnpp":
                fea2 = self.gcns[1](fea1 + xX, adj_low, adj_high, adj_low_unnormalized)
            else:
                fea2 = self.gcns[1](fea1, adj_low, adj_high, adj_low_unnormalized)
        return fea2

In [17]:
model  = GCN(nfeat=2,
        nhid=2,
        nclass=2,
        nlayers=2,
        nnodes=7,
        dropout=0.2,
        model_type='acmgcn',
        structure_info=0,
        variant=False,
        init_layers_X=1,)

model

GCN(
  (gcns): ModuleList(
    (0): GraphConvolution (2 -> 2)
    (1): GraphConvolution (2 -> 2)
  )
  (mlps): ModuleList()
)

In [18]:
from torch_geometric.utils.convert import to_scipy_sparse_matrix

import torch
import numpy as np
import scipy.sparse as sp

def normalize_tensor(mx, eqvar = None):
    """Row-normalize sparse matrix"""
    mx = sp.csr_matrix(mx)
    rowsum = np.array(mx.sum(1))
    if eqvar:
        r_inv = np.power(rowsum, -1.0/eqvar).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv, 0)
        mx = r_mat_inv.dot(mx)    
    else:
        r_inv = np.power(rowsum, -1.0).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = sp.diags(r_inv, 0)
        mx = r_mat_inv.dot(mx)
    return mx

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [19]:
n = 7
x = torch.Tensor([[1,0],[1,0],[1,0],[0,1],[0,1],[0,1],[0,1]])
y = torch.LongTensor([0,0,0, 1, 1, 1, 1])
edge_index = torch.LongTensor([[1,2],[1,4],[1,5],[2,1],[3,6],[3,7],[4,5],[4,1],[4,6],[4,7],[5,1],[5,4],[5,6],[6,3],[6,4],[6,5],[6,7],[7,3],[7,4],[7,6]]).T
edge_index = edge_index-1

In [20]:
x.to(device)
adj_low_unnormalized = to_scipy_sparse_matrix(edge_index)
adj_low = normalize_tensor(sp.identity(n) + adj_low_unnormalized)
adj_high = sp.identity(n) - adj_low
adj_low = sparse_mx_to_torch_sparse_tensor(adj_low).to(device)
adj_high = sparse_mx_to_torch_sparse_tensor(adj_high).to(device)
adj_low_unnormalized = sparse_mx_to_torch_sparse_tensor(adj_low_unnormalized).to(device)

In [21]:
model(x.to(device), adj_low.to(device), adj_high.to(device), adj_low_unnormalized.to(device))

tensor([[0.0798, 0.0580],
        [0.1047, 0.0512],
        [0.4791, 0.0000],
        [0.0218, 0.0143],
        [0.0808, 0.0567],
        [0.1062, 0.0703],
        [0.1523, 0.1012]], device='cuda:0', grad_fn=<MulBackward0>)